# Proyecto Final - Deep Learning (1INF52)
## Entregable 2: Fine-tuning de T5 para generación de resúmenes

**Integrantes:** *

Aarón Ulises Santillán Huamán - 20200445

> Este cuaderno implementa el pipeline de fine-tuning del modelo T5 sobre el dataset CNN/DailyMail
usando la librería 🤗 Hugging Face. Incluye carga de datos, preprocesamiento, entrenamiento,
evaluación con métricas ROUGE y guardado del modelo.

## 1. Instalación de dependencias

In [ ]:
# Si estás en Google Colab, descomenta y ejecuta esta celda una sola vez.
!pip install -q transformers datasets accelerate rouge-score evaluate sentencepiece


## 2. Importaciones y configuración inicial

In [ ]:
import os
import numpy as np
from datasets import load_dataset
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
)
import evaluate
import torch

# Verificar dispositivo disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Dispositivo: {device}")

# Fijar semilla para reproducibilidad
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)


## 3. Carga del dataset CNN/DailyMail

Usaremos la versión `3.0.0` del dataset `cnn_dailymail` disponible en Hugging Face Datasets.

In [ ]:
# Cargar el dataset CNN/DailyMail
dataset = load_dataset('cnn_dailymail', '3.0.0')
dataset

## 4. Carga del tokenizer y del modelo T5

In [ ]:
# Puedes cambiar 't5-small' por 't5-base' si tienes suficientes recursos
model_name = 't5-small'

tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device);


## 5. Preprocesamiento de datos
Convertimos cada ejemplo en pares `(input_ids, labels)` para el modelo T5.

- Entrada: prefijo `summarize: ` + artículo
- Salida: `highlights` (resumen humano)


In [ ]:
max_input_length = 512   # longitud máxima del artículo
max_target_length = 128  # longitud máxima del resumen

def preprocess_function(batch):
    # Prefijo de tarea para T5
    inputs = ["summarize: " + doc for doc in batch['article']]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding='max_length',
    )

    # Tokenización de los resúmenes (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch['highlights'],
            max_length=max_target_length,
            truncation=True,
            padding='max_length',
        )

    # Reemplazar tokens de padding con -100 para que no contribuyan a la pérdida
    labels_ids = labels['input_ids']
    labels_ids = [
        [(lid if lid != tokenizer.pad_token_id else -100) for lid in label]
        for label in labels_ids
    ]
    model_inputs['labels'] = labels_ids
    return model_inputs


In [ ]:
# Para reducir tiempos de prueba, puedes usar un subconjunto del dataset
small_train = dataset['train'].shuffle(seed=seed).select(range(20000))  # ajustar según recursos
small_val = dataset['validation'].shuffle(seed=seed).select(range(2000))

tokenized_train = small_train.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names,
)
tokenized_val = small_val.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['validation'].column_names,
)
tokenized_train, tokenized_val

## 6. Definición de la métrica ROUGE

In [ ]:
rouge = evaluate.load('rouge')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    return preds, labels

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Reemplazar -100 por el token de padding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
    )
    # Promediar y multiplicar por 100 para dejar en porcentaje
    result = {k: round(v * 100, 2) for k, v in result.items()}
    return result


## 7. Configuración de entrenamiento (TrainingArguments + Trainer)

In [ ]:
output_dir = './t5_cnn_summarization'

batch_size = 4  # ajustar según memoria de GPU/CPU
num_epochs = 3

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


## 8. Entrenamiento del modelo

In [ ]:
train_result = trainer.train()
trainer.save_model(output_dir)
train_result

## 9. Evaluación en el conjunto de validación

In [ ]:
metrics = trainer.evaluate()
metrics

## 10. Ejemplos de resúmenes generados

In [ ]:
def generate_summary(article, max_new_tokens=128):
    # Construimos el prompt para T5
    input_text = "summarize: " + article
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        truncation=True,
        max_length=max_input_length,
    ).to(device)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Tomamos un ejemplo del conjunto de validación original (no tokenizado)
example = dataset['validation'][0]
print('=== ARTÍCULO ORIGINAL ===')
print(example['article'][:1000], '...')

print('\n=== RESUMEN HUMANO (highlights) ===')
print(example['highlights'])

print('\n=== RESUMEN GENERADO POR EL MODELO ===')
generated = generate_summary(example['article'])
print(generated)


## 11. (Opcional) Publicar el modelo en Hugging Face Hub

In [ ]:
# Ejecutar esta celda solo si quieres subir tu modelo al Hub.
# 1) Primero: !huggingface-cli login (en la terminal/colab)
# 2) Luego define tu nombre de repo y ejecuta.

from huggingface_hub import notebook_login
print('Si estás en Colab/Jupyter, puedes autenticarte con: notebook_login()')
print('Luego usa: trainer.push_to_hub("nombre-usuario/t5-cnn-summarization")')
